# Functions 

In [1]:
def spring_force(force, p1, p2, equil=1.75):
    'Calculates stretch and returns force vector'
    stretch = np.linalg.norm(p2-p1)-equil
    unit_vector = (p2-p1)/np.linalg.norm(p2-p1)
    hooke_force = force*stretch # applies Hooke's Law
    return hooke_force*unit_vector
    
def pos_diff(p1,p2):
    'Returns tuple of difference vector between p1 and p2, and the magnitude of the distance'
    stretch = sqrt((p2[0] - p1[0])**2 + (p2[1]-p1[1])**2 + (p2[2] - p1[2])**2)
    diff = p2-p1
    return diff

# Initialization of Variables

In [2]:
%matplotlib qt
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import pylab
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

force = 5 # sample spring constant
np.set_printoptions(precision=4)
hook_offset = 0.375

E = np.array([0,0,6.735])
F = np.array([0,7,6.735])
G = np.array([9,7,6.375])
H = np.array([9,0.375,6.75])
I = np.array([7,-1.25,11])
J = np.array([-0.5,2.375,11.25])
K = np.array([6,7.125,11.5])

#Variable Positions- Standard Equilibrium
A = np.array([2.75,2.125,6.25])
B = np.array([2.625, 6,6.25])
C = np.array([6.125,6,6.25])
D = np.array([6.25,3,6.25])

#Variable Positions- With added forces
A_prime = np.array([2.75,1.15, 7.75])
B_prime = np.array([2,4.75,8.25])
C_prime = np.array([5.75,4.875,8.25])
D_prime = np.array([6,1.625,8])

#Variable Positions- With replacement wrench
A2_prime = np.array([2.25,2.125,7.5])
B2_prime = np.array([2.25,6.125,7.25])
C2_prime = np.array([6,6,7.25])
D2_prime = np.array([6,2,7.5])

# Calculation of Replacement Wrench 

In [3]:
#Forces causing new equilibrium
f1 = spring_force(force, C_prime, K)
f2 = spring_force(force, A_prime, J)
f3 = spring_force(force, D_prime, I)
sum_forces = f1+f2+f3

#print("sum forces:",sum_forces)
#Find unit vector for later moment projection
unit_force = sum_forces/sqrt(sum_forces[0]**2 + sum_forces[1]**2 + sum_forces[2]**2)

# Define point A_prime to be point of reference for wrench calculation; position vectors
r1 = C_prime - A_prime
r2 = A_prime - A_prime
r3 = D_prime - A_prime

# Moments 
m1 = np.cross(r1,f2)
m2 = np.cross(r2,f2)
m3 = np.cross(r3,f3)

#Calculate moment perpendicular
sum_moments = m1+m2+m3
m_parallel = np.dot(sum_moments, unit_force)*unit_force #moment projected onto unit force vector
m_perp = sum_moments - m_parallel
print("Parallel Moment:",np.linalg.norm(m_parallel))
print("Perpendicular Moment:", m_perp, np.linalg.norm(m_perp))


#solve with two equations, adjusted for absolute coordinates
x_loc = m_perp[1]/-sum_forces[2] 
y_loc = m_perp[0]/sum_forces[2]
#z_loc = A_prime[2] + z_function_adjustment
print("Replacement Location on Suspension:",(x_loc,y_loc))


#Z position adjustment
p1 = B_prime - A_prime
p2 = D_prime - A_prime
n = np.cross(p2,p1)
z = (n[0]*x_loc + n[1]*y_loc)/n[2]

#Find replacement location in absolute coordinates
inter_stretch = sum_forces/force/2 # divide by two for TWO springs applied
#print(inter_stretch)
r_start = (x_loc+A_prime[0],y_loc+A_prime[1],A_prime[2]-z + hook_offset)
r_end = (x_loc+A_prime[0]+inter_stretch[0], y_loc+A_prime[1] + inter_stretch[1], A_prime[2]-z + inter_stretch[2] + hook_offset)
#print(r_start)
#print(r_end)

Parallel Moment: 6.85816003973
Perpendicular Moment: [ 47.965  -67.5562  15.2435] 84.2428229083
Replacement Location on Suspension: (2.3122069911806311, 1.6416693988712212)


# Graph of Points

In [4]:
#Original Points without external forces
fig = pylab.figure()
ax = Axes3D(fig)
plt.title('Original Positions')
plt.xlabel('X')
plt.ylabel('Y')
index = 0
labels = ['A','B','C','D','E','F','G','H','I','J','K']
for point in [A,B,C,D,E,F,G,H,I,J,K]:
    ax.scatter3D(point[0], point[1], point[2])
    ax.text3D(point[0], point[1], point[2], '%s' % (labels[index]))
    index += 1
    
#Points after applied system of forces
fig2 = pylab.figure()
ax2 = Axes3D(fig2)
plt.title('System of Forces')
plt.xlabel('X')
plt.ylabel('Y')
index = 0
labels = ['A\'','B\'','C\'','D\'','E','F','G','H','I','J','K']
for point in [A_prime,B_prime,C_prime,D_prime,E,F,G,H,I,J,K]:
    ax2.scatter3D(point[0], point[1], point[2])
    ax2.text3D(point[0], point[1], point[2], '%s' % (labels[index]))
    index += 1
    
#Replacement position on suspension
ax2.scatter3D(r_start[0], r_start[1], r_start[2], c='r')
ax2.text3D(r_start[0], r_start[1], r_start[2], '{:.4}, {:.4}, {:.4}'.format(r_start[0],r_start[1],r_start[2]))

#Replacement end point
ax2.scatter3D(r_end[0], r_end[1], r_end[2], c='r')
ax2.text3D(r_end[0], r_end[1], r_end[2], '{:.4}, {:.4}, {:.4}'.format(r_end[0],r_end[1],r_end[2]))

#Replacement string line
#ax2.plot(np.array([r_start[0],r_end[0]]),np.array([r_start[1],r_end[1]]),np.array([r_start[2],r_end[2]]))

# Error Checking 

In [5]:
#Takes difference between system of forces position and replacement force position
e1 = pos_diff(A, A_prime) - pos_diff(A,A2_prime)
e2 = pos_diff(B, B_prime) - pos_diff(B,B2_prime)
e3 = pos_diff(C, C_prime) - pos_diff(C,C2_prime)
e4 = pos_diff(D, D_prime) - pos_diff(D,D2_prime)
print("Error 1:",e1)
print("Error 2:",e2)
print("Error 3:",e3)
print("Error 4:",e4)

Error 1: [ 0.5   -0.975  0.25 ]
Error 2: [-0.25  -1.375  1.   ]
Error 3: [-0.25  -1.125  1.   ]
Error 4: [ 0.    -0.375  0.5  ]


In [6]:
#Plots suspension positions next to each  other for visual analysis
fig3 = pylab.figure()
ax3 = Axes3D(fig3)
plt.xlim(0,11.5)
plt.ylim(0,11.5)
ax3.set_zlim3d(0,12)
#System of forces
index = 0
labels = ['A\'','B\'','C\'','D\'']
for point in [A_prime,B_prime,C_prime,D_prime]:
    ax3.scatter3D(point[0], point[1], point[2], c='b')
    ax3.text3D(point[0], point[1], point[2], '{}'.format(labels[index]))
    index += 1
    
#Replacement force
index = 0
labels2 = ['A2','B2','C2','D2']
for point in [A2_prime,B2_prime,C2_prime,D2_prime]:
    ax3.scatter3D(point[0], point[1], point[2],c='r')
    ax3.text3D(point[0], point[1], point[2], '{}'.format(labels2[index]))
    index += 1